## **Import Libraries**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.utils.data as data_utils
from torch.autograd import Variable
import math
import torchvision.models as models
from PIL import Image
from deepfool import deepfool
from deepfool_targeted import deepfool_targeted
import os

import numpy as np
import torch as torch
import copy
from torch.autograd.gradcheck import zero_gradients

import torch
import torchvision.datasets as datasets
from torchvision import transforms

import matplotlib.image as mpimg

import json
# import time

Please, check the `deepfool_targeted.py` file to see if the needed function is commented out.

In [ ]:
# Select image and the target label. Here the target variable stores the index of the label from " synset_words.txt "
im_orig = Image.open( 'test_im1.jpg' )
target = 298

## **ResNet50 (ImageNet)**

In [ ]:
net = models.resnet50( pretrained=True )

# Switch to evaluation mode
net.eval()


mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]


im = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])(im_orig)

# r: Total perturbation
# loop_i: Number of iterations required to fool the network
# label_orig: Original label of the image
# label_pert: Target label
# pert_image: Image after addition of perturbation
# confidences: A list containing confidence score for every iterations
r, loop_i, label_orig, label_pert, pert_image, confidences = deepfool_targeted(im, net, target)

labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

str_label_orig = labels[np.int_(label_orig)].split(',')[0]
str_label_pert = labels[np.int_(label_pert)].split(',')[0]

print("Original label = ", str_label_orig)
print("Perturbed label = ", str_label_pert)

def clip_tensor(A, minv, maxv):
    A = torch.max(A, minv*torch.ones(A.shape))
    A = torch.min(A, maxv*torch.ones(A.shape))
    return A

clip = lambda x: clip_tensor(x, 0, 255)

tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
transforms.Lambda(clip),
transforms.ToPILImage(),
transforms.CenterCrop(224)])

In [ ]:
# Plot original image, perturbation, perturbed image

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))


# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))

# Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)

# Display the original image with its label
axs[0].imshow(im)
axs[0].set_title(str_label_orig)


axs[1].imshow(tf(pert_image.cpu()[0]))
axs[1].set_title(str_label_pert)

# Display only the perturbations
r = np.squeeze(r)  # remove the first dimension
r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)

# Display the perturbed image with its label
axs[2].imshow(np.abs(r))
axs[2].set_title('Perturbation')

# # Display only the perturbations amplified
# r = np.squeeze(r)
# r = np.transpose(r, (1, 2, 0))
# amplification_factor = 5
# r_amplified = amplification_factor * r
# r_amplified = np.clip(r_amplified, 0, 1)
# axs[2].imshow(r_amplified)
# axs[2].set_title('Perturbation Mask (Amplified)')

plt.tight_layout()
plt.show()


In [ ]:
# Plot Confidence scores for perturbed image
plt.figure(figsize=(8, 6))
plt.plot(range(len(confidences)), confidences, color='blue', linestyle='--')
plt.title("Change in Target Class Confidence") 
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Target Class Confidence", fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12)
plt.show()


## **AlexNet (ImageNet)**

In [ ]:
net = models.alexnet(pretrained=True)

# Switch to evaluation mode
net.eval()

mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]



im = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])(im_orig)

# r: Total perturbation
# loop_i: Number of iterations required to fool the network
# label_orig: Original label of the image
# label_pert: Target label
# pert_image: Image after addition of perturbation
# confidences: A list containing confidence score for every iterations
r, loop_i, label_orig, label_pert, pert_image, confidences = deepfool_targeted(im, net, target)

labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

str_label_orig = labels[np.int_(label_orig)].split(',')[0]
str_label_pert = labels[np.int_(label_pert)].split(',')[0]

print("Original label = ", str_label_orig)
print("Perturbed label = ", str_label_pert)

def clip_tensor(A, minv, maxv):
    A = torch.max(A, minv*torch.ones(A.shape))
    A = torch.min(A, maxv*torch.ones(A.shape))
    return A

clip = lambda x: clip_tensor(x, 0, 255)

tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
transforms.Lambda(clip),
transforms.ToPILImage(),
transforms.CenterCrop(224)])

In [ ]:
# Plot original image, perturbation, perturbed image

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))


# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))

# Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)

# Display the original image with its label
axs[0].imshow(im)
axs[0].set_title(str_label_orig)


axs[1].imshow(tf(pert_image.cpu()[0]))
axs[1].set_title(str_label_pert)

# Display only the perturbations
r = np.squeeze(r)  # remove the first dimension
r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)

# Display the perturbed image with its label
axs[2].imshow(np.abs(r))
axs[2].set_title('Perturbation')

# # Display only the perturbations amplified
# r = np.squeeze(r)
# r = np.transpose(r, (1, 2, 0))
# amplification_factor = 5
# r_amplified = amplification_factor * r
# r_amplified = np.clip(r_amplified, 0, 1)
# axs[2].imshow(r_amplified)
# axs[2].set_title('Perturbation Mask (Amplified)')

plt.tight_layout()
plt.show()


In [ ]:
# Plot Confidence scores for perturbed image
plt.figure(figsize=(8, 6))
plt.plot(range(len(confidences)), confidences, color='blue', linestyle='--')
plt.title("Change in Target Class Confidence") 
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Target Class Confidence", fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12)
plt.show()


## **EfficientNetV2 (ImageNet)**

In [ ]:
net = models.efficientnet_v2_s(pretrained=True)

# Switch to evaluation mode
net.eval()

mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]



im = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])(im_orig)

# r: Total perturbation
# loop_i: Number of iterations required to fool the network
# label_orig: Original label of the image
# label_pert: Target label
# pert_image: Image after addition of perturbation
# confidences: A list containing confidence score for every iterations
r, loop_i, label_orig, label_pert, pert_image, confidences = deepfool_targeted(im, net, target)

labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

str_label_orig = labels[np.int_(label_orig)].split(',')[0]
str_label_pert = labels[np.int_(label_pert)].split(',')[0]

print("Original label = ", str_label_orig)
print("Perturbed label = ", str_label_pert)

def clip_tensor(A, minv, maxv):
    A = torch.max(A, minv*torch.ones(A.shape))
    A = torch.min(A, maxv*torch.ones(A.shape))
    return A

clip = lambda x: clip_tensor(x, 0, 255)

tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
transforms.Lambda(clip),
transforms.ToPILImage(),
transforms.CenterCrop(224)])

In [ ]:
# Plot original image, perturbation, perturbed image

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))


# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))

# Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)

# Display the original image with its label
axs[0].imshow(im)
axs[0].set_title(str_label_orig)


axs[1].imshow(tf(pert_image.cpu()[0]))
axs[1].set_title(str_label_pert)

# Display only the perturbations
r = np.squeeze(r)  # remove the first dimension
r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)

# Display the perturbed image with its label
axs[2].imshow(np.abs(r))
axs[2].set_title('Perturbation')

# # Display only the perturbations amplified
# r = np.squeeze(r)
# r = np.transpose(r, (1, 2, 0))
# amplification_factor = 5
# r_amplified = amplification_factor * r
# r_amplified = np.clip(r_amplified, 0, 1)
# axs[2].imshow(r_amplified)
# axs[2].set_title('Perturbation Mask (Amplified)')

plt.tight_layout()
plt.show()


In [ ]:
# Plot Confidence scores for perturbed image
plt.figure(figsize=(8, 6))
plt.plot(range(len(confidences)), confidences, color='blue', linestyle='--')
plt.title("Change in Target Class Confidence") 
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Target Class Confidence", fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12)
plt.show()


## **GoogleNet (ImageNet)**

In [ ]:
net = models.googlenet(pretrained=True)

# Switch to evaluation mode
net.eval()

mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]



im = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])(im_orig)

# r: Total perturbation
# loop_i: Number of iterations required to fool the network
# label_orig: Original label of the image
# label_pert: Target label
# pert_image: Image after addition of perturbation
# confidences: A list containing confidence score for every iterations
r, loop_i, label_orig, label_pert, pert_image, confidences = deepfool_targeted(im, net, target)

labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

str_label_orig = labels[np.int_(label_orig)].split(',')[0]
str_label_pert = labels[np.int_(label_pert)].split(',')[0]

print("Original label = ", str_label_orig)
print("Perturbed label = ", str_label_pert)

def clip_tensor(A, minv, maxv):
    A = torch.max(A, minv*torch.ones(A.shape))
    A = torch.min(A, maxv*torch.ones(A.shape))
    return A

clip = lambda x: clip_tensor(x, 0, 255)

tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
transforms.Lambda(clip),
transforms.ToPILImage(),
transforms.CenterCrop(224)])

In [ ]:
# Plot original image, perturbation, perturbed image

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))


# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))

# Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)

# Display the original image with its label
axs[0].imshow(im)
axs[0].set_title(str_label_orig)


axs[1].imshow(tf(pert_image.cpu()[0]))
axs[1].set_title(str_label_pert)

# Display only the perturbations
r = np.squeeze(r)  # remove the first dimension
r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)

# Display the perturbed image with its label
axs[2].imshow(np.abs(r))
axs[2].set_title('Perturbation')

# # Display only the perturbations amplified
# r = np.squeeze(r)
# r = np.transpose(r, (1, 2, 0))
# amplification_factor = 5
# r_amplified = amplification_factor * r
# r_amplified = np.clip(r_amplified, 0, 1)
# axs[2].imshow(r_amplified)
# axs[2].set_title('Perturbation Mask (Amplified)')

plt.tight_layout()
plt.show()


In [ ]:
# Plot Confidence scores for perturbed image
plt.figure(figsize=(8, 6))
plt.plot(range(len(confidences)), confidences, color='blue', linestyle='--')
plt.title("Change in Target Class Confidence") 
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Target Class Confidence", fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12)
plt.show()


## **InceptionV3 (ImageNet)**

In [ ]:
net = models.inception_v3(pretrained=True)

# Switch to evaluation mode
net.eval()

mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]



im = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])(im_orig)

# r: Total perturbation
# loop_i: Number of iterations required to fool the network
# label_orig: Original label of the image
# label_pert: Target label
# pert_image: Image after addition of perturbation
# confidences: A list containing confidence score for every iterations
r, loop_i, label_orig, label_pert, pert_image, confidences = deepfool_targeted(im, net, target)

labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

str_label_orig = labels[np.int_(label_orig)].split(',')[0]
str_label_pert = labels[np.int_(label_pert)].split(',')[0]

print("Original label = ", str_label_orig)
print("Perturbed label = ", str_label_pert)

def clip_tensor(A, minv, maxv):
    A = torch.max(A, minv*torch.ones(A.shape))
    A = torch.min(A, maxv*torch.ones(A.shape))
    return A

clip = lambda x: clip_tensor(x, 0, 255)

tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
transforms.Lambda(clip),
transforms.ToPILImage(),
transforms.CenterCrop(224)])

In [ ]:
# Plot original image, perturbation, perturbed image

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))


# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))

# Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)

# Display the original image with its label
axs[0].imshow(im)
axs[0].set_title(str_label_orig)


axs[1].imshow(tf(pert_image.cpu()[0]))
axs[1].set_title(str_label_pert)

# Display only the perturbations
r = np.squeeze(r)  # remove the first dimension
r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)

# Display the perturbed image with its label
axs[2].imshow(np.abs(r))
axs[2].set_title('Perturbation')

# # Display only the perturbations amplified
# r = np.squeeze(r)
# r = np.transpose(r, (1, 2, 0))
# amplification_factor = 5
# r_amplified = amplification_factor * r
# r_amplified = np.clip(r_amplified, 0, 1)
# axs[2].imshow(r_amplified)
# axs[2].set_title('Perturbation Mask (Amplified)')

plt.tight_layout()
plt.show()


In [ ]:
# Plot Confidence scores for perturbed image
plt.figure(figsize=(8, 6))
plt.plot(range(len(confidences)), confidences, color='blue', linestyle='--')
plt.title("Change in Target Class Confidence") 
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Target Class Confidence", fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12)
plt.show()


## **VisionTransformer (ImageNet)**

In [ ]:
net = models.vit_b_16(pretrained=True)

# Switch to evaluation mode
net.eval()

mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]



im = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])(im_orig)

# r: Total perturbation
# loop_i: Number of iterations required to fool the network
# label_orig: Original label of the image
# label_pert: Target label
# pert_image: Image after addition of perturbation
# confidences: A list containing confidence score for every iterations
r, loop_i, label_orig, label_pert, pert_image, confidences = deepfool_targeted(im, net, target)

labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

str_label_orig = labels[np.int_(label_orig)].split(',')[0]
str_label_pert = labels[np.int_(label_pert)].split(',')[0]

print("Original label = ", str_label_orig)
print("Perturbed label = ", str_label_pert)

def clip_tensor(A, minv, maxv):
    A = torch.max(A, minv*torch.ones(A.shape))
    A = torch.min(A, maxv*torch.ones(A.shape))
    return A

clip = lambda x: clip_tensor(x, 0, 255)

tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
transforms.Lambda(clip),
transforms.ToPILImage(),
transforms.CenterCrop(224)])

In [ ]:
# Plot original image, perturbation, perturbed image

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))


# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))

# Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)

# Display the original image with its label
axs[0].imshow(im)
axs[0].set_title(str_label_orig)


axs[1].imshow(tf(pert_image.cpu()[0]))
axs[1].set_title(str_label_pert)

# Display only the perturbations
r = np.squeeze(r)  # remove the first dimension
r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)

# Display the perturbed image with its label
axs[2].imshow(np.abs(r))
axs[2].set_title('Perturbation')

# # Display only the perturbations amplified
# r = np.squeeze(r)
# r = np.transpose(r, (1, 2, 0))
# amplification_factor = 5
# r_amplified = amplification_factor * r
# r_amplified = np.clip(r_amplified, 0, 1)
# axs[2].imshow(r_amplified)
# axs[2].set_title('Perturbation Mask (Amplified)')

plt.tight_layout()
plt.show()


In [ ]:
# Plot Confidence scores for perturbed image
plt.figure(figsize=(8, 6))
plt.plot(range(len(confidences)), confidences, color='blue', linestyle='--')
plt.title("Change in Target Class Confidence") 
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Target Class Confidence", fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12)
plt.show()


# **Run test on a single image**

In [ ]:
models_list = [models.resnet50(pretrained=True),
               models.alexnet(pretrained=True),
               models.efficientnet_v2_s(pretrained=True),
               models.googlenet(pretrained=True),
               models.inception_v3(pretrained=True),
               models.vit_b_16(pretrained=True)]

results_dict = {}

for i, net in enumerate(models_list):
    # Switch to evaluation mode
    net.eval()

    im_orig = Image.open('./images/test_images/test_im4.jpeg')
    target = 413

    mean = [ 0.485, 0.456, 0.406 ]
    std = [ 0.229, 0.224, 0.225 ]

    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = mean,
                             std = std)])(im_orig)

    r, loop_i, label_orig, label_pert, pert_image, confidences = deepfool_targeted(im, net, target)

    # Store the results in a dictionary
    results_dict[i] = {
        "r": r,
        "loop_i": loop_i,
        "label_orig": label_orig,
        "label_pert": label_pert,
        "pert_image": pert_image,
        "confidences": confidences
    }

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

    str_label_orig = labels[np.int_(label_orig)].split(',')[0]
    str_label_pert = labels[np.int_(label_pert)].split(',')[0]

    print("Original label for model {} = {}".format(i+1, str_label_orig))
    print("Perturbed label for model {} = {}".format(i+1, str_label_pert))

    def clip_tensor(A, minv, maxv):
        A = torch.max(A, minv*torch.ones(A.shape))
        A = torch.min(A, maxv*torch.ones(A.shape))
        return A

    clip = lambda x: clip_tensor(x, 0, 255)

    tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
    transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
    transforms.Lambda(clip),
    transforms.ToPILImage(),
    transforms.CenterCrop(224)])
    
    ## Run the following code if you want to save the images ##

    # Save the original image
    # Convert the tensor to a numpy array
    im = im.numpy().transpose((1, 2, 0))

    # # Denormalize the image
    im = std * im + mean
    im = np.clip(im, 0, 1)
    # # Convert the NumPy array to a PIL image object
    im_pil = Image.fromarray(np.uint8(im * 255))

    # # Save the original image
    im_pil.save(f"original_{i}.jpg")

    # Save the perturbed image
    pert_image = tf(pert_image.cpu()[0])
    pert_image.save(f"perturbed_{i}.jpg")

    # # Save perturbations
    # r = np.squeeze(r)  # remove the first dimension
    # r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)
    # r = np.abs(r)
    # r_pil = Image.fromarray(np.uint8(r * 255))
    # r_pil.save(f"perturbation_{i}.jpg")

    # Save perturbations amplified
    r = np.squeeze(r)
    r = np.transpose(r, (1, 2, 0))
    amplification_factor = 20
    r_amplified = amplification_factor * r
    r_amplified = np.abs(r_amplified)
    r_pil = Image.fromarray(np.uint8(r_amplified * 255))
    r_pil.save(f"perturbation_amplified{i}.jpg")

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'

# Plot the confidence scores for each perturbed image
plt.figure(figsize=(8, 6), dpi=300)
for i in range(6):
    confidences = results_dict[i]["confidences"]
    color = ['blue', 'red', 'green', 'orange', 'purple', 'gray'][i % 6]
    linestyle = ['--', ':', '-.', '--', ':', '-.'][i % 6]
    label = ["ResNet50", "AlexNet", "EfficientNetV2", "GoogLeNet", "InceptionV3", "Vision Transformer"][i % 6]
    plt.plot(range(len(confidences)), confidences, color=color, linestyle=linestyle, label=label)

plt.title("Change in Target Class Confidence ({} to {})".format(str_label_orig, str_label_pert)) 
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Target Class Confidence", fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12)
plt.legend(fontsize=12, loc='center left', bbox_to_anchor=(0.6, 0.8))
plt.show()


# **DeepFool vs Targeted DeepFool**

## **DeepFool**

In [ ]:
im_orig = Image.open('test_im1.jpg')

In [ ]:
net = models.resnet50(pretrained=True)

# Switch to evaluation mode
net.eval()

mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]


im = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])(im_orig)


r, loop_i, label_orig, label_pert, pert_image = deepfool(im, net)

labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

str_label_orig = labels[np.int_(label_orig)].split(',')[0]
str_label_pert = labels[np.int_(label_pert)].split(',')[0]

print("Original label = ", str_label_orig)
print("Perturbed label = ", str_label_pert)

def clip_tensor(A, minv, maxv):
    A = torch.max(A, minv*torch.ones(A.shape))
    A = torch.min(A, maxv*torch.ones(A.shape))
    return A

clip = lambda x: clip_tensor(x, 0, 255)

tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
transforms.Lambda(clip),
transforms.ToPILImage(),
transforms.CenterCrop(224)])

## Run following codes if you want to save the images ##

# Save the original image
# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))
# # Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)
# # Convert the NumPy array to a PIL image object
im_pil = Image.fromarray(np.uint8(im * 255))
# # Save the original image
im_pil.save(f"original.jpg")

# Save the perturbed image
pert_image = tf(pert_image.cpu()[0])
pert_image.save(f"perturbed.jpg")

# # Save perturbations
# r = np.squeeze(r)  # remove the first dimension
# r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)
# r = np.abs(r)
# r_pil = Image.fromarray(np.uint8(r * 255))
# r_pil.save(f"perturbation.jpg")

# Save amplified perturbations
r = np.squeeze(r)
r = np.transpose(r, (1, 2, 0))
amplification_factor = 20
r_amplified = amplification_factor * r
r_amplified = np.abs(r_amplified)
r_pil = Image.fromarray(np.uint8(r_amplified * 255))
r_pil.save(f"perturbation(amplified).jpg")

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

# Display the original image with its label
# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))

# Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)

axs[0].imshow(im)
axs[0].set_title(str_label_orig)

# Display the perturbed image with its label
axs[1].imshow(tf(pert_image.cpu()[0]))
axs[1].set_title(str_label_pert)

# # Display only the perturbations
# r = np.squeeze(r)  # remove the first dimension
# r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)
# axs[2].imshow(np.abs(r))
# axs[2].set_title('Perturbation')

# Display only the perturbations amplified
r = np.squeeze(r)
r = np.transpose(r, (1, 2, 0))
amplification_factor = 10
r_amplified = amplification_factor * r
r_amplified = np.clip(r_amplified, 0, 1)
axs[2].imshow(r_amplified)
axs[2].set_title('Perturbation Mask (Amplified)')

plt.tight_layout()
plt.show()


## **Targeted DeepFool**

In [ ]:
im_orig = Image.open('test_im1.jpg')
target = 298

In [ ]:
net = models.resnet50(pretrained=True)

# Switch to evaluation mode
net.eval()

mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]


im = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])(im_orig)

r, loop_i, label_orig, label_pert, pert_image, confidences = deepfool_targeted(im, net, target)

labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

str_label_orig = labels[np.int_(label_orig)].split(',')[0]
str_label_pert = labels[np.int_(label_pert)].split(',')[0]

print("Original label = ", str_label_orig)
print("Perturbed label = ", str_label_pert)

def clip_tensor(A, minv, maxv):
    A = torch.max(A, minv*torch.ones(A.shape))
    A = torch.min(A, maxv*torch.ones(A.shape))
    return A

clip = lambda x: clip_tensor(x, 0, 255)

tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
transforms.Lambda(clip),
transforms.ToPILImage(),
transforms.CenterCrop(224)])

## Run following codes if you want to save the images ##

# Save the original image
# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))
# # Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)
# # Convert the NumPy array to a PIL image object
im_pil = Image.fromarray(np.uint8(im * 255))
# # Save the original image
im_pil.save(f"original.jpg")
# Save the perturbed image
pert_image = tf(pert_image.cpu()[0])
pert_image.save(f"perturbed.jpg")

# # Save perturbations
# r = np.squeeze(r)  # remove the first dimension
# r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)
# r = np.abs(r)
# r_pil = Image.fromarray(np.uint8(r * 255))
# r_pil.save(f"perturbation.jpg")

# Save perturbations amplified
r = np.squeeze(r)
r = np.transpose(r, (1, 2, 0))
amplification_factor = 20
r_amplified = amplification_factor * r
r_amplified = np.abs(r_amplified)
r_pil = Image.fromarray(np.uint8(r_amplified * 255))
r_pil.save(f"perturbation(amplified).jpg")

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

# Display the original image with its label
# Convert the tensor to a numpy array
im = im.numpy().transpose((1, 2, 0))

# Denormalize the image
im = std * im + mean
im = np.clip(im, 0, 1)

axs[0].imshow(im)
axs[0].set_title(str_label_orig)

# Display the perturbed image with its label
axs[1].imshow(tf(pert_image.cpu()[0]))
axs[1].set_title(str_label_pert)

# # Display only the perturbations
# r = np.squeeze(r)  # remove the first dimension
# r = np.transpose(r, (1, 2, 0))  # transpose the dimensions to (H, W, 3)
# axs[2].imshow(np.abs(r))
# axs[2].set_title('Perturbation')

# Display only the perturbations amplified
r = np.squeeze(r)
r = np.transpose(r, (1, 2, 0))
amplification_factor = 10
r_amplified = amplification_factor * r
r_amplified = np.clip(r_amplified, 0, 1)
axs[2].imshow(r_amplified)
axs[2].set_title('Perturbation Mask (Amplified)')

plt.tight_layout()
plt.show()


In [ ]:
# Plot Confidence scores for perturbed image
plt.figure(figsize=(8, 6))
plt.plot(range(len(confidences)), confidences, color='blue', linestyle='--')
plt.title("Change in Target Class Confidence") 
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Target Class Confidence", fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12)
plt.show()
